<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/BTC_AAI_LLM_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai -q
!pip install pykrakenapi -q
!pip install krakenex -q

In [7]:
import os
import json
import random
import re # Import regex module
from datetime import datetime
from google.colab import userdata
import google.generativeai as genai

# Import krakenex for Kraken API interaction
import krakenex
# No need for pykrakenapi for this simple public query.

# --- Load API Keys from Colab Secrets ---
GOOGLE_API_KEY = None
KRAKEN_API_KEY = None
KRAKEN_API_SECRET = None

try:
    GOOGLE_API_KEY = userdata.get('GEMINI')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Gemini API configured successfully.")
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    print("Please ensure you have set the 'GEMINI' secret in Google Colab.")
    print("This demonstration will proceed with a simulated LLM if the API is not available.")

try:
    KRAKEN_API_KEY = userdata.get('KRAKEN_API_KEY')
    KRAKEN_API_SECRET = userdata.get('KRAKEN_API_SECRET')
    if not (KRAKEN_API_KEY and KRAKEN_API_SECRET):
        print("Kraken API keys not found in secrets. Bitcoin market data will be simulated.")
except Exception as e:
    print(f"Error loading Kraken API keys: {e}")
    print("Bitcoin market data will be simulated.")


# --- 1. LLM (Large Language Model) Capability ---
class LLM:
    def __init__(self, model_name="gemini-1.5-pro-latest"):
        self.model_name = model_name
        self.client = None
        if GOOGLE_API_KEY:
            try:
                self.client = genai.GenerativeModel(model_name)
                print(f"Connected to real LLM: {self.model_name}")
            except Exception as e:
                print(f"Could not initialize real Gemini model: {e}. Falling back to simulated LLM.")
                self.client = None # Fallback if model initialization fails

    def _extract_json_from_markdown(self, text):
        """
        Extracts a JSON string from a markdown code block (```json ... ```).
        If no markdown block is found, it attempts to find the outermost JSON object.
        """
        # First, try to find content within ```json ... ```
        match = re.search(r"```json\s*(.*?)\s*```", text, re.DOTALL)
        if match:
            return match.group(1).strip()

        # If no markdown block, try to find the first '{' and last '}'
        start_brace = text.find('{')
        end_brace = text.rfind('}')
        if start_brace != -1 and end_brace != -1 and end_brace > start_brace:
            return text[start_brace : end_brace + 1].strip()

        # If neither, return the original text, likely to cause JSONDecodeError
        return text.strip()


    def analyze_market_sentiment(self, data, query):
        """
        Uses a real LLM (Gemini) or simulates one to analyze market data
        and provide a suggested action and analysis.
        """
        print(f"\n--- LLM ({self.model_name}) Analysis ---")

        if self.client:
            # Construct a detailed prompt for the LLM
            # Emphasize ONLY JSON output to minimize markdown wrapping
            prompt = f"""
            You are an expert cryptocurrency market analyst. Your task is to analyze the provided Bitcoin market data and a user query, then provide a concise market outlook and a clear action suggestion.

            **Market Data:**
            {json.dumps(data, indent=2)}

            **User Query:** "{query}"

            **Instructions:**
            1.  Provide a brief market outlook based on the current price and any historical context provided (e.g., 24h change).
            2.  Suggest one of the following actions: "BUY", "SELL", or "HOLD".
            3.  **Your ENTIRE response MUST be a VALID JSON string, and nothing else.** Do NOT include any introductory text, prose, conversational filler, or markdown formatting (e.g., no ```json block, just the JSON object itself).
                Example of desired output: {{"suggestion": "BUY", "analysis": "Bitcoin shows strong bullish momentum..."}}
            4.  Keep the "analysis" concise, around 2-3 sentences.
            """
            try:
                response = self.client.generate_content(prompt)
                llm_output_text = response.text.strip()
                print(f"Raw LLM Output:\n{llm_output_text}")

                # Attempt to extract JSON from markdown, then parse
                clean_llm_output = self._extract_json_from_markdown(llm_output_text)
                # Ensure the string is not empty before parsing
                if not clean_llm_output:
                    raise ValueError("LLM output was empty or could not be extracted.")

                parsed_output = json.loads(clean_llm_output)

                suggestion = parsed_output.get("suggestion", "HOLD").upper()
                analysis = parsed_output.get("analysis", "Could not fully analyze the market.")

            except (json.JSONDecodeError, ValueError) as e:
                print(f"JSON Parsing Error: {e}. Attempted to parse: '{clean_llm_output[:100]}...'")
                print("Falling back to rule-based suggestion due to LLM parsing error.")
                # Fallback if LLM output is not valid JSON even after stripping markdown
                price_change_24h = data['current_price'] - data.get('past_price_24h', data['current_price'])
                if price_change_24h > data['current_price'] * 0.01: # More than 1% up
                    suggestion = "BUY"
                    analysis = "Rule-based: Significant positive price change observed."
                elif price_change_24h < -data['current_price'] * 0.01: # More than 1% down
                    suggestion = "SELL"
                    analysis = "Rule-based: Significant negative price change observed."
                else:
                    suggestion = "HOLD"
                    analysis = "Rule-based: Price relatively stable."
            except Exception as e:
                print(f"An unexpected error occurred with LLM: {e}. Falling back to rule-based suggestion.")
                # General fallback for other LLM related errors
                price_change_24h = data['current_price'] - data.get('past_price_24h', data['current_price'])
                if price_change_24h > data['current_price'] * 0.01:
                    suggestion = "BUY"
                    analysis = "Rule-based: Significant positive price change observed."
                elif price_change_24h < -data['current_price'] * 0.01:
                    suggestion = "SELL"
                    analysis = "Rule-based: Significant negative price change observed."
                else:
                    suggestion = "HOLD"
                    analysis = "Rule-based: Price relatively stable."
        else:
            # Simulated LLM logic if real LLM is not available
            print("Using simulated LLM (real LLM not configured or failed to load).")
            price_change_24h = data['current_price'] - data.get('past_price_24h', data['current_price'])
            if price_change_24h > data['current_price'] * 0.01:
                suggestion = "BUY"
                analysis = "Simulated: Significant positive price change observed."
            elif price_change_24h < -data['current_price'] * 0.01:
                suggestion = "SELL"
                analysis = "Simulated: Significant negative price change observed."
            else:
                suggestion = "HOLD"
                analysis = "Simulated: Price relatively stable."

        print(f"LLM's suggested action: {suggestion}")
        print(f"LLM's analysis: {analysis}")
        return suggestion, analysis

# --- 2. Bitcoin Data Bot (Fetches Real Data from Kraken if API Keys are Available) ---
class BitcoinDataBot:
    def __init__(self, api_key=None, api_secret=None):
        self.kraken_api = None
        if api_key and api_secret:
            try:
                self.kraken_api = krakenex.API(key=api_key, secret=api_secret)
                print("Kraken API client initialized for data fetching.")
            except Exception as e:
                print(f"Error initializing Kraken API client: {e}. Will simulate data.")
                self.kraken_api = None
        else:
            print("Kraken API keys not provided. Data will be simulated.")


    def get_market_data(self, pair="XBTUSD"): # XBTUSD is Bitcoin/USD on Kraken
        """
        Fetches Bitcoin market data from Kraken or simulates it.
        """
        print(f"\n--- Bitcoin Data Bot (Tool) ---")
        data = {}

        if self.kraken_api:
            try:
                kraken_pair = "XXBTZUSD" # Standard Kraken symbol for BTC/USD
                ticker_response = self.kraken_api.query_public('Ticker', {'pair': kraken_pair})
                # --- IMPORTANT: Uncomment this line to see the raw response if it fails again ---
                # print(f"Kraken Ticker Raw Response for {kraken_pair}: {json.dumps(ticker_response, indent=2)}")

                # Check for Kraken's 'error' array first. It's empty on success.
                if 'error' in ticker_response and ticker_response['error']: # Check if 'error' key exists and array is NOT empty
                    error_message = ticker_response['error'][0] if ticker_response['error'] else "Unknown Kraken API error."
                    print(f"Kraken Ticker API Error: {error_message}. Simulating data.")
                    data = self._simulate_market_data(pair) # Fallback to simulated data
                elif 'result' in ticker_response and kraken_pair in ticker_response['result']:
                    ticker_info = ticker_response['result'][kraken_pair]
                    current_price = float(ticker_info['c'][0]) # Last trade closed price
                    past_price_24h = float(ticker_info['o']) # 24-hour rolling open price

                    data = {
                        "currency_pair": kraken_pair,
                        "current_price": current_price,
                        "past_price_24h": past_price_24h,
                        "timestamp_est": datetime.now().strftime("%Y-%m-%d %H:%M:%S EST"),
                        "source": "Kraken API"
                    }
                    print(f"Fetched real market data from Kraken: {json.dumps(data, indent=2)}")
                else:
                    # This else block means no 'error' and either no 'result' or 'result' without the pair
                    print(f"Unexpected or empty 'result' in Kraken Ticker API for {kraken_pair}. Simulating data.")
                    data = self._simulate_market_data(pair)
            except Exception as e:
                print(f"Failed to fetch real data from Kraken API (general exception): {e}. Simulating data.")
                data = self._simulate_market_data(pair)
        else:
            print("Kraken API not initialized or keys not found. Simulating data.")
            data = self._simulate_market_data(pair)

        return data

    def _simulate_market_data(self, pair):
        # Simulate dynamic price data to allow for different LLM responses
        current_price = round(random.uniform(35000, 70000), 2) # Wider range for better variation
        price_change_magnitude = random.uniform(500, 4000) * random.choice([-1, 1, 0]) # More variability
        past_price_24h = round(current_price - price_change_magnitude, 2)
        # Ensure past_price_24h doesn't go negative or too low unrealistically
        if past_price_24h <= 0:
            past_price_24h = current_price * 0.95 # Fallback to a reasonable percentage

        return {
            "currency_pair": pair.upper(),
            "current_price": current_price,
            "past_price_24h": past_price_24h,
            "timestamp_est": datetime.now().strftime("%Y-%m-%d %H:%M:%S EST"),
            "source": "Simulated Data"
        }

# --- 3. Simulate Agentic AI (Decision Making & Action Orchestration) ---
class CryptoTradingAgent:
    def __init__(self, llm_instance, data_bot_instance):
        self.llm = llm_instance
        self.data_bot = data_bot_instance
        self.portfolio = {"BTC": 0.5, "USD": 10000} # Starting portfolio
        print("\n--- Agentic AI Initialized ---")
        print(f"Initial Portfolio: {self.portfolio}")

    def execute_transaction(self, action, amount_usd, price_btc_usd):
        """
        Simulates executing a Bitcoin transaction.
        In a real scenario, this would involve exchange API calls and wallet management
        using private Kraken API endpoints, which require specific API key permissions.
        """
        print(f"\n--- Transaction Simulator Bot (Part of Agent's Tools) ---")
        transaction_id = f"TXN-{random.randint(10000, 99999)}"
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S EST") # Using EST as per user preference

        if action == "BUY":
            btc_bought = amount_usd / price_btc_usd
            if self.portfolio["USD"] >= amount_usd:
                self.portfolio["USD"] -= amount_usd
                self.portfolio["BTC"] += btc_bought
                status = "SUCCESS"
                print(f"Simulated BUY order: Bought {btc_bought:.4f} BTC for ${amount_usd:.2f} at ${price_btc_usd:.2f}/BTC")
            else:
                status = "FAILED"
                print(f"Simulated BUY order FAILED: Insufficient USD funds to buy ${amount_usd:.2f} worth of BTC.")
        elif action == "SELL":
            btc_sold = amount_usd / price_btc_usd # Amount of BTC to sell to get desired USD
            # Simple check: Ensure we have at least 0.001 BTC to sell to avoid tiny sells
            if self.portfolio["BTC"] >= btc_sold and btc_sold >= 0.001:
                self.portfolio["USD"] += amount_usd
                self.portfolio["BTC"] -= btc_sold
                status = "SUCCESS"
                print(f"Simulated SELL order: Sold {btc_sold:.4f} BTC for ${amount_usd:.2f} at ${price_btc_usd:.2f}/BTC")
            else:
                status = "FAILED"
                print(f"Simulated SELL order FAILED: Insufficient BTC funds or amount too small to sell {btc_sold:.4f} BTC.")
        else:
            status = "N/A"
            print(f"No transaction executed for action: {action}")

        transaction_record = {
            "transaction_id": transaction_id,
            "action": action,
            "amount_usd": amount_usd,
            "btc_price_at_txn": price_btc_usd,
            "status": status,
            "timestamp": timestamp,
            "updated_portfolio": self.portfolio
        }
        print(f"Transaction Record: {json.dumps(transaction_record, indent=2)}")
        return transaction_record

    def process_market_query(self, query):
        """
        Orchestrates the entire process:
        1. Uses the Data Bot to get market data.
        2. Passes data to the LLM for analysis and suggestion.
        3. Decides on an action based on LLM's suggestion and portfolio.
        4. Uses the Transaction Simulator Bot to "execute" if applicable.
        """
        print(f"\n===== Agentic AI Processing Query: '{query}' =====")

        # Step 1: Agent uses Data Bot (Tool) to get current market data
        market_data = self.data_bot.get_market_data()
        current_btc_price = market_data['current_price']

        # Step 2: Agent sends data to LLM for analysis and suggested action
        llm_suggestion, llm_analysis_details = self.llm.analyze_market_sentiment(market_data, query)
        print(f"LLM Suggested Action: {llm_suggestion}")

        # Step 3: Agentic decision-making based on LLM's suggestion and internal rules
        action_to_take = "HOLD" # Default action
        amount_to_trade_usd = 0

        if llm_suggestion == "BUY":
            # Agent's rule: If LLM suggests BUY and we have enough USD, buy 1000 USD worth of BTC
            if self.portfolio["USD"] >= 1000:
                action_to_take = "BUY"
                amount_to_trade_usd = 1000
                print(f"Agent decided to '{action_to_take}' {amount_to_trade_usd} USD worth of BTC based on LLM suggestion and available funds.")
            else:
                print("Agent decided to HOLD: Insufficient USD funds to buy, despite LLM's BUY suggestion.")
        elif llm_suggestion == "SELL":
            # Agent's rule: If LLM suggests SELL and we have enough BTC, sell 0.05 BTC (or what's available if less)
            target_btc_to_sell = 0.05
            if self.portfolio["BTC"] >= 0.001: # Ensure we have *some* BTC to consider selling
                btc_to_sell = min(target_btc_to_sell, self.portfolio["BTC"])
                if btc_to_sell * current_btc_price >= 100: # Ensure minimum trade value
                    action_to_take = "SELL"
                    amount_to_trade_usd = btc_to_sell * current_btc_price
                    print(f"Agent decided to '{action_to_take}' {btc_to_sell:.4f} BTC ({amount_to_trade_usd:.2f} USD) based on LLM suggestion and available BTC.")
                else:
                    print("Agent decided to HOLD: Sell amount too small, despite LLM's SELL suggestion.")
            else:
                print("Agent decided to HOLD: No BTC in portfolio to sell.")
        else:
            print("Agent decided to HOLD as per LLM's neutral suggestion or internal rules.")

        # Step 4: Agent uses Transaction Simulator Bot (Tool) to execute the decided action
        if action_to_take in ["BUY", "SELL"] and amount_to_trade_usd > 0:
            self.execute_transaction(action_to_take, amount_to_trade_usd, current_btc_price)
        else:
            print("No actionable transaction initiated by the agent.")

        print(f"===== Agentic AI Processing Complete. Current Portfolio: {self.portfolio} =====")
        return self.portfolio

# --- Main Demonstration ---
if __name__ == "__main__":
    # Initialize the core components
    llm = LLM(model_name='gemini-1.5-pro-latest')
    data_bot = BitcoinDataBot(api_key=KRAKEN_API_KEY, api_secret=KRAKEN_API_SECRET)
    crypto_agent = CryptoTradingAgent(llm, data_bot)

    print("\nStarting the Bitcoin, Bot, Agentic AI, LLM Synergy Demo (Gemini 1.5)")
    print("------------------------------------------------------------------\n")

    # Scenario 1: A general query, leading to a potential action
    print("\n--- Scenario 1: User asks for general market outlook ---")
    crypto_agent.process_market_query("What's the current outlook for Bitcoin?")

    # Scenario 2: Another query, simulating different market conditions and outcome
    print("\n--- Scenario 2: User asks about Bitcoin's recent performance ---")
    crypto_agent.process_market_query("How has Bitcoin been performing in the last 24 hours?")

    # Scenario 3: User asks for a quick investment tip ---
    print("\n--- Scenario 3: User asks for a quick investment tip ---")
    crypto_agent.process_market_query("Should I invest in Bitcoin right now?")

    print("\nDemonstration End. Remember this is a simplified simulation.")

Google Gemini API configured successfully.
Connected to real LLM: gemini-1.5-pro-latest
Kraken API client initialized for data fetching.

--- Agentic AI Initialized ---
Initial Portfolio: {'BTC': 0.5, 'USD': 10000}

Starting the Bitcoin, Bot, Agentic AI, LLM Synergy Demo (Gemini 1.5)
------------------------------------------------------------------


--- Scenario 1: User asks for general market outlook ---

===== Agentic AI Processing Query: 'What's the current outlook for Bitcoin?' =====

--- Bitcoin Data Bot (Tool) ---
Fetched real market data from Kraken: {
  "currency_pair": "XXBTZUSD",
  "current_price": 109118.6,
  "past_price_24h": 108932.0,
  "timestamp_est": "2025-07-09 01:09:29 EST",
  "source": "Kraken API"
}

--- LLM (gemini-1.5-pro-latest) Analysis ---
Raw LLM Output:
```json
{"suggestion": "HOLD", "analysis": "Bitcoin shows slight upward movement with a 0.17% increase in the last 24 hours.  Given the minimal change, it's advisable to hold current positions and monitor ma